# 图像曝光增强
功能介绍：使用模型对曝光不足的输入图片进行HDR效果增强。    
样例输入：png图像。     
样例输出：增强后png图像。  
## 前期准备  
基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：
1. 获取模型和测试数据：我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，将模型和测试数据下载到本地，将image_HDR_enhance.pb放入model目录，将测试图片放入data目录。

2. 模型转换：进入model目录，执行：  
    ```shell
    atc --model=./image_HDR_enhance.pb --framework=3 --output=image_HDR_enhance --soc_version=Ascend310B1 --input_shape="input:1,512,512,3" --input_format=NHWC --output_type=FP32
    ```

    其中各个参数具体含义如下：  
    * --model：原始模型文件。
    * --framework：原始框架类型，0:Caffe; 1:MindSpore; 2:Tensorflow; 5:Onnx。
    * --output：离线推理om模型文件路径。
    * --soc_version：昇腾AI处理器型号，填写"Ascend310B1"。
    * --input_shape：模型输入节点名称和shape。
    * --input_format：输入Tensor的内存排列方式。
    * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型。

## 模型推理实现  
1.导入需要的第三方库以及AscendCL推理所需文件。 

In [ ]:
import sys
import cv2
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import acl

import acllite_utils as utils
import constants as constants
from acllite_model import AclLiteModel
from acllite_resource import resource_list

2.定义资源管理类，包括初始化acl、释放acl资源功能。  

In [ ]:
class AclLiteResource:
    """
    AclLiteResource类
    """
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """
        初始化资源
        """
        print("init resource stage:")
        ret = acl.init() # acl初始化

        ret = acl.rt.set_device(self.device_id) # 指定运算的device
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id) # 创建context
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream() # 创建stream
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode() # 获取运行模式
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream) # 销毁stream

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context) # 释放context

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id) # 释放device
        
        print("Release acl resource success")

3.实现具体推理功能，包含预处理、推理、后处理等功能。  

In [ ]:
from download import download

# 获取模型om文件
model_url = "https://modelers.cn/coderepo/web/v1/file/MindSpore-Lab/cluoud_obs/main/media/examples/mindspore-courses/orange-pi-mindspore/04-HDR/image_HDR_enhance.zip"
download(model_url, "./model", kind="zip", replace=True)

In [ ]:
path = os.getcwd()
input_w = 512   # 推理输入width
input_h = 512   # 推理输入height
INPUT_DIR = os.path.join(path, 'data/') # 输入路径
OUTPUT_DIR = os.path.join(path, 'out/') # 输出路径

def pre_process(dir_name, input_h, input_w):
    """
    预处理
    """
    BGR = cv2.imread(dir_name).astype(np.float32)
    h = BGR.shape[0]
    w = BGR.shape[1]
    # 进行归一化、缩放、颜色转换
    BGR = BGR / 255.0
    BGR = cv2.resize(BGR, (input_h, input_w))
    RGB = cv2.cvtColor(BGR, cv2.COLOR_BGR2RGB)
    return RGB, h, w

def post_process(input_img, result_list, pic, input_h, input_w):
    """
    后处理
    """
    o_w, o_h = input_img.shape[:2]
    # 获取推理结果，进行形状变换
    data = result_list[0].reshape(input_h, input_w, 3)
    # 进行缩放、颜色转换
    output = (cv2.resize(data, (o_w, o_h)) * 255.0).astype(np.uint8)
    output_img = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    # 保存增强后图像
    file_name = os.path.join(OUTPUT_DIR, pic)
    cv2.imwrite(file_name, output_img)
    # 拼接输入图像和增强后图像，返回进行显示
    BGR_U8 = np.concatenate([input_img, output_img], axis=1)
    return BGR_U8

def main():
    # 创建推理结果存放路径
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    # acl初始化
    acl_resource = AclLiteResource()
    acl_resource.init()
    # 加载模型
    model_path = os.path.join(path, "model/image_HDR_enhance.om")
    model = AclLiteModel(model_path)
    # 遍历数据集进行推理
    src_dir = os.listdir(INPUT_DIR)
    for pic in src_dir:
        if not pic.lower().endswith(('.bmp', '.dib', '.png', '.jpg', '.jpeg', '.pbm', '.pgm', '.ppm', '.tif', '.tiff')):
            print('it is not a picture, %s, ignore this file and continue,' % pic)
            continue
        pic_path = os.path.join(INPUT_DIR, pic)
        input_img = cv2.imread(pic_path)
        # 进行预处理
        RGB_image, o_h, o_w = pre_process(pic_path, input_h, input_w)
        # 计算推理耗时
        start_time = time.time()
        # 执行推理
        result_list = model.execute([RGB_image, ])
        end_time = time.time()
        # 打印推理的图片信息和耗时
        print('pic:{}'.format(pic))
        print('pic_size:{}x{}'.format(o_h, o_w))
        print('time:{}ms'.format(int((end_time - start_time) * 1000)))
        print('\n')
        # 进行后处理
        img_result = post_process(input_img, result_list, pic, input_h, input_w)      
        # 显示输入图像和增强后图像
        img_RGB = img_result[:, :, [2, 1, 0]] # RGB
        plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img_RGB)
        plt.show()

4.运行样例    

In [ ]:
main()

5.运行完成上述样例后，打印出测试图像名称和尺寸、推理时间、资源初始化和释放信息。并展示了输入图像和增强后图像，以及将增强后图像保存在out目录下，可进行查看。  

## 样例总结
样例实现流程包括几个关键步骤：  
1.初始化acl资源：在调用acl相关资源时，必须先初始化AscendCL，否则可能会导致后续系统内部资源初始化出错。  
2.对输入进行预处理：包括图像归一化、缩放、颜色转换操作。  
3.推理：利用AclLiteModel.execute接口进行推理。    
4.对推理结果进行后处理：包括形状变换、缩放、颜色转换操作，保存增强后图像。  
5.可视化图片：利用plt将结果画出。